<a href="https://colab.research.google.com/github/kitsamho/imdb_app/blob/main/Simple_Transformers_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers==3.0.2

     |████████████████████████████████| 769 kB 6.6 MB/s 
     |████████████████████████████████| 3.0 MB 52.0 MB/s 
     |████████████████████████████████| 1.2 MB 62.6 MB/s 
     |████████████████████████████████| 895 kB 53.8 MB/s 


In [2]:
!pip install simpletransformers

     |████████████████████████████████| 221 kB 9.5 MB/s 
     |████████████████████████████████| 124 kB 16.4 MB/s 
     |████████████████████████████████| 43 kB 2.7 MB/s 
     |████████████████████████████████| 8.0 MB 17.4 MB/s 
     |████████████████████████████████| 2.6 MB 58.3 MB/s 
     |████████████████████████████████| 1.6 MB 56.8 MB/s 
     |████████████████████████████████| 264 kB 67.6 MB/s 
     |████████████████████████████████| 636 kB 52.1 MB/s 
     |████████████████████████████████| 3.3 MB 52.0 MB/s 
     |████████████████████████████████| 97 kB 7.6 MB/s 
     |████████████████████████████████| 170 kB 59.6 MB/s 
     |████████████████████████████████| 133 kB 64.9 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 118 kB 64.7 MB/s 
     |████████████████████████████████| 243 kB 63.2 MB/s 
     |████████████████████████████████| 111 kB 50.1 MB/s 
     |████████████████████████████████| 75 kB 4.4 MB/s 
     |█████████████████

In [3]:
# Importing stock ml libraries
import warnings
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import DistilBertTokenizer, DistilBertModel
import logging
logging.basicConfig(level=logging.ERROR)

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
df = pd.read_json('/content/drive/MyDrive/tmdb_data_main.json')

In [8]:
df = df[['tmdb_id','movie','overview','genres','genres_new','popularity','release_year','vote_average','budget','revenue','poster_path']]
df = df.dropna(subset=['genres'])




In [9]:
genre_include = ['Action', 'Adventure', 'Comedy',  'Drama', 'Horror', 'Romance', 'Science Fiction', 'Thriller']

In [10]:
def genres_check(film_genres):
    
    for genre in film_genres:
        if genre not in genre_include:
            return False
    return True

def multi_label_prep(film_genres):
    prep = []
    for genre in genre_include:
        if genre in film_genres:
            prep.append(1)
        else:
            prep.append(0)
            
    return prep

df['genres_valid'] = df['genres'].apply(genres_check)
df = df[df.genres_valid ==True]
df.shape
genre_include = list(genre_include)
genre_include.sort()
genre_include



df['labels'] = df['genres'].apply(multi_label_prep)

In [11]:
df = pd.concat([df[['overview']],df[['labels']]['labels'].apply(pd.Series)],axis=1)

In [12]:
df.columns = ['overview']+genre_include
df

,overview,Action,Adventure,Comedy,Drama,Horror,Romance,Science Fiction,Thriller
0,Jack Torrance accepts a caretaker job at the O...,0,0,0,0,1,0,0,1
4,"In a near-future Britain, young Alexander DeLa...",0,0,0,1,0,0,1,0
5,Camp counselors are stalked and murdered by an...,0,0,0,0,1,0,0,0
6,A Coca-Cola bottle dropped from an airplane ra...,1,0,1,0,0,0,0,0
7,"George Lutz and his wife Kathleen, move into t...",0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...
19507,A secret government agency who recruits the mo...,1,0,1,0,1,0,0,0
19513,When his girlfriend is forced to marry another...,1,0,0,1,0,1,0,0
19592,"After years of estrangement, an embittered you...",0,0,0,1,1,0,0,1
19606,Tom and Janet have been happily married for ye...,0,0,1,0,0,1,0,1


In [13]:
df['labels'] = list(zip(df.Action.tolist(), df.Adventure.tolist(),  
                        df.Comedy.tolist(),  df.Drama.tolist(),
                        df.Horror.tolist(), df.Romance.tolist(),  
                        df['Science Fiction'].tolist(), df.Thriller.tolist(),
                        
                        ))
df['text'] = df['overview']

In [14]:
from sklearn.model_selection import train_test_split


train_df, eval_df = train_test_split(df, test_size=0.2)

In [15]:
train_df[['labels','text']]

,labels,text
4103,"(0, 0, 1, 0, 0, 1, 0, 0)","“Cásese quien pueda”, is a romantic comedy pla..."
2476,"(1, 0, 0, 0, 0, 0, 0, 0)",Fong Sai Yuk's uninhibited arrogance toward a ...
7114,"(0, 0, 1, 1, 0, 1, 0, 0)",Three tales of very different women using thei...
17464,"(0, 0, 0, 1, 0, 0, 0, 0)",A hitchhiking woman is abducted by a young cou...
19593,"(0, 0, 1, 0, 0, 1, 0, 0)",Gab de la Cuesta is a high-strung career woman...
...,...,...
13305,"(0, 0, 1, 0, 0, 1, 0, 0)","When her husband abruptly ends their marriage,..."
18402,"(0, 0, 1, 1, 0, 0, 0, 0)",Five best friends from college reunite 10 year...
6238,"(0, 0, 0, 1, 0, 1, 0, 0)",A sailor prone to violent outbursts is sent to...
5802,"(0, 0, 1, 1, 0, 0, 0, 0)",After 11-year-old Zachary Cowan strikes his cl...


In [16]:
from simpletransformers.classification import MultiLabelClassificationModel, ClassificationModel

In [17]:
import torch


cuda_available = torch.cuda.is_available()



In [18]:
cuda_available

True

In [20]:
model = MultiLabelClassificationModel('distilbert', 'distilbert-base-uncased', num_labels=8, args={'train_batch_size':2, 'gradient_accumulation_steps':16, 'learning_rate': 3e-5, 'num_train_epochs': 3, 'max_seq_length': 512},use_cuda=True)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForMultiLabelSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForMultiLabelSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bia

In [21]:
model.train_model(train_df[['labels','text']])

  0%|          | 0/7957 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/3979 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/3979 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/3979 [00:00<?, ?it/s]

(744, 0.3378983017727507)

In [22]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

  0%|          | 0/1990 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/249 [00:00<?, ?it/s]

In [23]:
result


{'LRAP': 0.8514749142538072, 'eval_loss': 0.3159459102225591}

In [24]:
test = ['During the WWII, acclaimed Polish musician Wladyslaw faces various struggles as he loses contact with his family. As the situation worsens, he hides in the ruins of Warsaw in order to survive.']

In [25]:
# to_predict = test_df.comment_text.apply(lambda x: x.replace('\n', ' ')).tolist()
preds, outputs = model.predict(test)

sub_df = pd.DataFrame(outputs, columns=genre_include)


# sub_df['id'] = test_df['id']
# sub_df = sub_df[['id', 'toxic','severe_toxic','obscene','threat','insult','identity_hate']]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
sub_df

,Action,Adventure,Comedy,Drama,Horror,Romance,Science Fiction,Thriller
0,0.056854,0.041138,0.030914,0.924316,0.073853,0.051178,0.024277,0.394287
